In [1]:
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from pandas import Series
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Activation, Dense
from keras.layers import LSTM
from keras.layers import Dropout
# Use scikit-learn to grid search the batch size and epochs
import numpy
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

/Users/yjlee/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [10]:
class HyperParams:
    def __init__(self, num_net, num_unit, epoch, batch_num, rmse, test_value, predict_value):
        self.num_net = num_net
        self.num_unit = num_unit
        self.epoch = epoch
        self.batch_num = batch_num
        self.rmse = rmse
        self.test_value = test_value
        self.predict_value = predict_value   

hyperParams_list = []
result_list = {"num_net":[], "num_unit":[], "epoch":[], "batch_size":[], "rmse":[], "test_value":[], "predict_value":[]}  

In [16]:
# load dataset
dataset = read_csv('data/data_second.csv', header=0, index_col=0)
values = dataset.values
col_num = len(values[1,:])
# ensure all data is float
values = values.astype('float32')
# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

###########shift_3
num_lag = 3
reframed = series_to_supervised(scaled, num_lag, 1)
drop_col_for = [i for i in range(col_num, num_lag*col_num,1)]
drop_col_back = [i for i in range(num_lag*col_num+1, (num_lag+1)*col_num, 1)]
drop_col_for.extend(drop_col_back)
drop_col = drop_col_for
reframed.drop(reframed.columns[drop_col], axis=1, inplace=True)
reframed.head()
values = reframed.values
total_length = len(values)
train_length = int(total_length*0.85)
train = values[:train_length, :]
test = values[train_length:, :]
# split into input and outputs
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]
valid_X, valid_y = test_X[0:len(test_X)-3,:], test_y[0:len(test_y)-3]

In [ ]:
######DNN model
num_unit = 200
neuron_num = 250
num_net = 3
#neuron_num_5 = 150
epoch=3000
batch_num = 30
for i in range(10):
    model = Sequential()
    if num_net == 2:
        model.add(Dense(neuron_num, input_dim=train_X.shape[1], activation = 'relu'))
        model.add(Dense(neuron_num, activation = 'relu'))
        model.add(Dense(1))
    elif num_net == 3:
        model.add(Dense(neuron_num, input_dim=train_X.shape[1], activation = 'relu'))
        model.add(Dense(neuron_num, activation = 'relu'))
        model.add(Dense(neuron_num, activation = 'relu'))
        model.add(Dense(1))
    else:
        model.add(Dense(neuron_num, input_dim=train_X.shape[1], activation = 'relu'))
        #model.add(Dropout(0.25))
        model.add(Dense(neuron_num, activation = 'relu'))
        #model.add(Dropout(0.25))
        model.add(Dense(neuron_num, activation = 'relu'))
        model.add(Dense(neuron_num, activation = 'relu'))
        model.add(Dense(1))
    model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
    # fit network
    history = model.fit(train_X, train_y, epochs=epoch, batch_size=batch_num, validation_data=(valid_X, valid_y), verbose=2, shuffle=True)
    num_prediction = 3
    yhat = model.predict(test_X)
    # invert scaling for forecast
    inv_yhat = concatenate((yhat, test_X[:, 1:]), axis = 1)
    inv_yhat = scaler.inverse_transform(inv_yhat)
    inv_yhat = inv_yhat[:,0]
    # invert scaling for actual
    test_y = test_y.reshape((len(test_y), 1))
    inv_y = concatenate((test_y, test_X[:, 1:]), axis = 1)
    inv_y = scaler.inverse_transform(inv_y)
    inv_y = inv_y[:,0]

    test_array = inv_yhat[len(inv_y)-num_prediction*2:len(inv_y)-num_prediction]
    test_value = str(test_array[0])+", "+str(test_array[1])+", "+str(test_array[2])
    # calculate RMSE
    rmse = sqrt(mean_squared_error(inv_y[len(inv_y)-num_prediction*2:len(inv_y)-num_prediction], inv_yhat[len(inv_y)-num_prediction*2:len(inv_y)-num_prediction]))
    predict_array = inv_yhat[len(inv_y)-num_prediction:len(inv_y)]
    predict_value = str(predict_array[0])+", "+str(predict_array[1])+", "+str(predict_array[2])
    print("#################################################################")
    print(num_net, epoch, batch_num, rmse, predict_value)
    #result_rmse_2[epoch_index, batch_index] = rmse
    hyperParams = HyperParams(num_net, num_unit, epoch, batch_num, rmse, test_value, predict_value)
    hyperParams_list.append(hyperParams)
    result_list["num_net"].append(num_net)
    result_list["num_unit"].append(num_unit)
    result_list["epoch"].append(epoch)
    result_list["batch_size"].append(batch_num)
    result_list["rmse"].append(rmse)
    result_list["test_value"].append(test_value)
    result_list["predict_value"].append(predict_value)
    i += 1


In [ ]:
result_df = DataFrame.from_dict(result_list)
result_df